<a href="https://colab.research.google.com/github/geoffroydautichamp/huggingface/blob/main/kaamelott_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!ls -d

.


In [ ]:
!pip install transformers

In [9]:
import json
import pandas as pd

dataset = pd.read_csv('all_scripts_summaries.csv', header=0, sep='|')
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: './all_scripts_summaries.csv'

In [ ]:
!pip install torch

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer

model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

# Charger les données
df = pd.read_csv('all_scripts_summaries.csv', sep='|')

# Convertir en dataset Hugging Face
dataset = Dataset.from_pandas(df)

model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    prompt = f"""
    Tu es un scénariste expert en dialogues humoristiques et médiévaux, et tu écris des dialogues pour la série Kaamelott à partir d'un brief.
    ###
    titre: {example['episode']}
    ###
    brief: {example['brief']}
    ###
    dialogue:
    """
    # Tokenize input
    input_encodings = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)

    # Tokenize labels, replacing padding tokens with -100 to avoid gradient updates on padding
    with tokenizer.as_target_tokenizer():
        label_encodings = tokenizer(example["dialogue"], padding="max_length", truncation=True, max_length=512)

    labels = label_encodings["input_ids"]

    # Replace padding token in labels with -100 (ignored by CrossEntropyLoss)
    labels = [tokenizer.pad_token_id if token != tokenizer.pad_token_id else -100 for token in labels]

    return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": labels
    }

tokenized_datasets = dataset.map(tokenize_function, remove_columns=['episode', 'brief', 'dialogue'])

Map:   0%|          | 0/89 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
tokenized_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 89
})

In [ ]:
for line in tokenized_datasets:
  print(len(line['input_ids']), len(line['labels']), len(line['attention_mask']))

512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 

In [ ]:
print(tokenized_datasets[1])

{'input_ids': [2740, 3, 15, 7, 73, 3, 7, 75, 15958, 52, 343, 15, 2205, 3, 35, 7478, 7, 13934, 343, 5125, 3, 15, 17, 3, 12986, 154, 208, 1724, 6, 3, 15, 17, 3, 17, 76, 27128, 159, 93, 7478, 7, 171, 50, 8187, 2209, 265, 15, 3171, 17, 3, 85, 5687, 3, 26, 31, 202, 4456, 5, 1713, 30345, 8349, 10, 13962, 1754, 325, 11860, 87, 18255, 1713, 30345, 4456, 10, 3039, 245, 3, 15145, 2134, 76, 15, 3, 8934, 20, 50, 4762, 146, 3, 17, 52, 10079, 29, 15, 6, 90, 3, 52, 32, 23, 13962, 6, 276, 2970, 6942, 159, 15, 3, 15, 17, 2846, 35, 23, 22491, 19736, 4669, 93, 3, 28476, 7456, 7, 3, 15, 17, 3, 12986, 154, 2165, 15, 7, 20, 20269, 171, 7236, 25061, 93, 11987, 3, 85, 73, 5063, 1238, 20, 3, 115, 154, 5756, 1417, 10379, 155, 3569, 5, 1713, 30345, 7478, 10, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
print(tokenized_datasets[0])

{'input_ids': [2740, 3, 15, 7, 73, 3, 7, 75, 15958, 52, 343, 15, 2205, 3, 35, 7478, 7, 13934, 343, 5125, 3, 15, 17, 3, 12986, 154, 208, 1724, 6, 3, 15, 17, 3, 17, 76, 27128, 159, 93, 7478, 7, 171, 50, 8187, 2209, 265, 15, 3171, 17, 3, 85, 5687, 3, 26, 31, 202, 4456, 5, 1713, 30345, 8349, 10, 20875, 26, 10398, 1753, 87, 18255, 1713, 30345, 4456, 10, 8410, 7, 3, 26, 31, 444, 20262, 28840, 171, 3, 16983, 50, 22268, 3386, 110, 3, 35, 900, 107, 19081, 7, 6, 13962, 6, 1915, 565, 2165, 3, 15, 17, 73, 4538, 26105, 972, 41, 1409, 12534, 3335, 17422, 7171, 61, 19736, 4669, 25846, 297, 244, 110, 979, 895, 77, 1724, 6, 971, 142, 21407, 295, 247, 245, 12413, 792, 15, 3, 85, 1137, 3, 26, 31, 20940, 7, 14589, 7, 20, 16283, 3, 15, 17, 16872, 6, 4007, 31, 85, 197, 546, 31, 1558, 2202, 10924, 260, 11503, 93, 3852, 12449, 303, 24845, 7, 3, 15, 17, 20440, 7, 5, 1713, 30345, 7478, 10, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
import os
import torch

# Ensure CPU mode
os.environ["CUDA_VISIBLE_DEVICES"] = ""
print(f"Using device: {torch.cuda.is_available()}")  # Should print False

#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

training_args = TrainingArguments(
    output_dir='content/output_dir',
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1,
    per_device_train_batch_size=1,  # Reduce batch size
    gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps
    bf16=True,
    no_cuda=True
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets
)

trainer.train()

# Save the fine-tuned model and tokenizer
trainer.save_model("content/output_dir")
tokenizer.save_pretrained("content/output_dir")

Using device: False


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1590: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


Step,Training Loss
1,37.520700


('content/output_dir/tokenizer_config.json',
 'content/output_dir/special_tokens_map.json',
 'content/output_dir/spiece.model',
 'content/output_dir/added_tokens.json',
 'content/output_dir/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Define the correct model path
model_path = "content/output_dir"

# Load the model and tokenizer correctly
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Detect device (CPU or GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Define the input
episode = "La fausse route"
brief = "Arthur et Perceval sont à cheval à travers la plaine. Perceval ne sait pas par où aller. Arthur perd patience. Ils croisent un paysan mais ne comprennent pas ses explications."

# Construct the prompt
input_text = f"""
Tu es un scénariste expert en dialogues humoristiques et médiévaux, et tu écris des dialogues pour la série Kaamelott à partir d'un brief.
Voici un brief et un titre pour un épisode de Kaamelott. Tu dois écrire un dialogue pour cet épisode, avec le ton et la structure propre à la série Kaamelott. Le dialogue doit être drôle et dans le style de la série Kaamelott.
###
titre: {episode}
###
brief: {brief}
###
dialogue:
"""

# Tokenize input and move tensors to the same device as the model
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)

# Generate output
model.eval()
with torch.no_grad():
    output_tokens = model.generate(
        **inputs,
        max_length=512,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

# Decode and print the generated dialogue
generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print("\n📝 Dialogue généré :\n")
print(generated_text)